This code was developed based on Google Colab

In [1]:
!pip install econml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: shap
    Found existing installation: shap 0.49.1
    Uninstalling shap-0.49.1:
      Successfully uninstalled shap-0.49.1


In [2]:
!pip install --upgrade pip
!pip install econml thinc --upgrade --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 35.2 MB/s  0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.3.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.6
    Uninstalling thinc-8.3.6:
      Successfully uninstalled thinc-8.3.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [thinc]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, but you have thinc 9.1.1 which is incompatible.


In [3]:
!pip install numpy==1.24.4 --force-reinstall
!pip install econml --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 19.3 MB/s  0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
ERROR: Failed to build 'numpy' when getting requirements to build wheel


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

import re
import seaborn as sns

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from econml.dml import CausalForestDML

In [6]:
## Read files and data clean
def read_satellite_data(satellite):
    # --- Main Execution ---
    df_satellite = pd.read_csv(f'/content/drive/My Drive/Colab Notebooks/Phenology/Satellite Code/data/tables/phenology_climate/{satellite}.csv')
    # print(df_satellite)

    veg_class = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Phenology/Satellite Code/data/tables/veg_class/veg_class.csv')
    df = pd.merge(df_satellite, veg_class, on=['latitude', 'longitude'], how='inner')
    df = df[df['veg_class'].isin([11, 12, 13, 14])]
    return df

In [ ]:
## Moving window analysis

In [7]:
def causal_ml(data, name, mv):
    feature_names = get_feature_names()
    years, windows = get_years_and_windows(name, mv)
    # print(windows)
    # print(years)
    all_effects = {}  # key: (lat, lon), value: dict of effects

    for i, (start, end) in enumerate(windows):
        print(f'Processing window {i + 1}')
        window_label = f"_es_w{i+1}"
        # middle_year = (start + end) // 2
        # window_label = f"_es_{middle_year}"

        for idx, row in data.iterrows():
            key = (row['latitude'], row['longitude'])

            # Compute effect for this window
            effect, pvalue = model_pixel_window(row, start, end, years, feature_names, name)

            # Initialize row dict if first time
            if key not in all_effects:
                all_effects[key] = {
                    'latitude': row['latitude'],
                    'longitude': row['longitude']
                }

            # Add each feature's effect to the row
            for fname, val, pval in zip(feature_names, effect, pvalue):
            # for fname, val in zip(['sos', 'summer_t', 'autumn_t'], effect):
                # print(fname, val)
                all_effects[key][f"{fname}{window_label}"] = val
                all_effects[key][f"{fname}{window_label}_p"] = pval

                # print('===============')
                # print(all_effects)
    # Convert to DataFrame
    df_effects = pd.DataFrame(list(all_effects.values()))
    return df_effects


In [8]:
from sklearn.linear_model import LinearRegression
from econml.dml import LinearDML

## Feature names
def get_feature_names():
    return ['sos', 'spring_t', 'summer_t', 'autumn_t',
            'spring_p', 'summer_p', 'autumn_p',
            'spring_r', 'summer_r', 'autumn_r', ]
## Modeling a single row-window
def model_pixel_window(row, start_year, end_year, years, feature_names, name):
    window_years = [y for y in years if start_year <= y <= end_year]

    df_long = pd.DataFrame({
        'year': window_years,
        'eos': [row[f'eos_{y}'] for y in window_years],
        'sos': [row[f'sos_{y}'] for y in window_years],
        **{f: [row[f'{f}_{y}'] for y in window_years] for f in feature_names if f != 'sos'}
    })
    # print(df_long)
    # PEP725 special filtering
    if name == 'pep725':
        df_long = df_long[df_long['eos'].notna() & df_long['sos'].notna()]
        if len(df_long) < 10:
            return np.full(len(feature_names), np.nan)
    # print(df_long)
    # Standardization
    for col in feature_names + ['eos']:
        # mean, std = df_long[col].mean(), df_long[col].std()
        # df_long[col] = (df_long[col] - mean) / std if std > 0 else 0.0
        min_val = df_long[col].min()
        max_val = df_long[col].max()
        if max_val > min_val:
            df_long[col] = 2 * (df_long[col] - min_val) / (max_val - min_val) - 1
        else:
            df_long[col] = 0.0  # Constant column

    # return cf.const_marginal_effect(X).mean(axis=0)
    # Loop over each feature as treatment
    effect_estimates = []
    pvalue_estimates = []
    for i, treat_var in enumerate(feature_names):
        test_var = True
        # if treat_var == "sos" or treat_var == "summer_t" or treat_var == "autumn_t":
        # if treat_var == "sos" or treat_var == "summer_t" or treat_var == "autumn_t" or treat_var == "spring_t" or treat_var == "spring_p" or treat_var == "summer_p" or treat_var == "autumn_p":
        if test_var == True:
            T = df_long[treat_var].to_numpy().ravel()
            Y = df_long['eos'].to_numpy().ravel()
            control_vars = [f for f in feature_names if f != treat_var]
            W = df_long[control_vars].to_numpy() if control_vars else np.ones((len(Y), 1))

            cf = LinearDML(
                model_y=LinearRegression(),  # OLS for Y
                model_t=LinearRegression(),  # OLS for T
                discrete_treatment=False,
                random_state=42,
                # cv=3,  # Fewer folds → faster
                # n_jobs=-1  # Uncomment to parallelize
            )


            try:
                cf.fit(Y, T, W=W)
                effect = cf.ate()
                ate_inf = cf.ate_inference()
                p_value = ate_inf.pvalue()

                # print(f"P-Value: {p_value}")

            except Exception as e:
                print(f"Error for {treat_var}: {e}")
                effect = np.nan
                p_value = np.nan

            effect_estimates.append(effect)
            pvalue_estimates.append(p_value)

    return np.array(effect_estimates), np.array(pvalue_estimates)

In [9]:
## Years & moving windows
def get_years_and_windows(name, mv):
    years = list(range(1982, 2017))
    # windows = [(1982 + i, 1998 + i) for i in range(19)] if mv else [(1982, 2017)]
    windows = [(start, start + 14) for start in range(1982, 2003)]

    # years_mv = [y for y in years if mv[0] <= y <= mv[1]]
    # windows_mv = [w for w in windows if mv[0] <= w[0] <= mv[1]]

    # print('years', years_mv)
    # print('windows', windows_mv)
    return years, windows

In [10]:
## Read avhrr data
satellite = "avhrr"
df = read_satellite_data(satellite)
print(df.shape)
# df = df.sample(frac=1, random_state=42).reset_index(drop=True).iloc[0:10]


(128264, 458)


In [11]:
mv = (1982, 1990) ## max 2002
import time
start_time = time.time()
df_chunk = df.iloc[115000:120000].reset_index(drop=True)

window_effect = causal_ml(df_chunk, satellite, mv)
print(window_effect.head())
window_effect.to_csv(f"/content/drive/My Drive/Colab Notebooks/Phenology/Figures Code/results/figure3/effect_size/avhrr/effect_size_ate_115000_120000.csv", index=False)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing window 1
Processing window 2
Processing window 3
Processing window 4
Processing window 5
Processing window 6
Processing window 7
Processing window 8
Processing window 9
Processing window 10
Processing window 11
Processing window 12
Processing window 13
Processing window 14
Processing window 15
Processing window 16
Processing window 17
Processing window 18
Processing window 19
Processing window 20
Processing window 21
   latitude  longitude  sos_es_w1   sos_es_w1_p  spring_t_es_w1  \
0    41.855    -90.645   0.573134  3.462380e-01        0.545301   
1    41.855    -90.545   0.882370  9.761960e-02        0.778944   
2    41.855    -90.445   0.488815  3.182866e-01        0.427557   
3    41.855    -90.045   1.530001  8.359566e-11        0.068206   
4    41.855    -89.945   1.249573  1.808117e-07        0.018571   

   spring_t_es_w1_p  summer_t_es_w1  summer_t_es_w1_p  autumn_t_es_w1  \
0          0.492932        0.215310      4.199361e-01       -0.554448   
1          0.256858